In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt


import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.download.snowex_lidar import download_dem

In [ ]:
import os
lidar_dir = '/Users/zachkeskinen/Documents/spicy-snow/data/lidar'

os.makedirs(lidar_dir, exist_ok = True)
os.chdir(lidar_dir)
download_dem()